# Feature generation of WoE

Weight of evidence (WoE) is a measure of how much the evidence supports or undermines a hypothesis. It measures the relative risk of an attribute of binning level. Since it evolved from credit scoring world, it is generally described as a measure of the separation of good and bad customers.

In [1]:
import sys
sys.path.append('../../../')
from python.algos.feature import quantile_bucket, woe_encode

import pyspark.sql.functions as F
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Init spark

In [2]:
import metaspore as ms

spark_confs={
        "spark.network.timeout":"500",
        "spark.sql.codegen.wholeStage": "false"
    }

spark_session = ms.spark.get_session(local=True, app_name='Feature generation WoE Demo', spark_confs=spark_confs)

/home/spark/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
22/06/10 07:52:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/10 07:52:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Load trainning dataset

In [3]:
train_dataset = spark_session.read.csv('s3://dmetasoul-bucket/demo/risk/tianchi/train.csv', header=True, inferSchema=False)
train_dataset.limit(10).toPandas()

22/06/10 07:52:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,Aug-2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000.0,5,18.49,461.9,D,D2,219843.0,5 years,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,May-2002,1723.0,1.0,None,None,None,None,10.0,None,None,None,None,None,13.0,None,None,None,None
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,May-2006,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,118000.0,1,2015-08-01,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,May-1999,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000.0,3,12.99,101.07,C,C2,54.0,None,1,29000.0,2,2016-03-01,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,Aug-1977,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0
5,5,11000.0,3,7.99,344.65,A,A5,51727.0,7 years,0,39000.0,2,2017-04-01,0,9,512.0,21,17.14,0.0,730.0,734.0,19.0,0.0,0.0,4047.0,31.1,52.0,1,0,Jul-1998,10.0,1.0,12.0,1.0,2.0,2.0,1.0,1.0,48.0,2.0,3.0,2.0,19.0,0.0,0.0,0.0,0.0
6,6,2050.0,3,7.69,63.95,A,A4,180083.0,9 years,0,35000.0,0,2014-10-01,0,0,517.0,14,17.49,0.0,755.0,759.0,12.0,0.0,0.0,3111.0,8.5,23.0,0,0,Oct-2006,0.0,1.0,0.0,1.0,3.0,3.0,7.0,11.0,3.0,10.0,18.0,3.0,12.0,0.0,0.0,0.0,3.0
7,7,11500.0,3,14.98,398.54,C,C3,214017.0,1 year,1,30000.0,2,2014-01-01,0,0,100.0,4,32.6,0.0,665.0,669.0,8.0,1.0,1.0,14021.0,59.7,33.0,1,0,Dec-1994,0.0,1.0,0.0,4.0,4.0,4.0,4.0,16.0,10.0,5.0,21.0,4.0,8.0,0.0,0.0,0.0,2.0
8,8,12000.0,3,12.99,404.27,C,C2,188.0,5 years,2,60000.0,1,2016-05-01,1,0,792.0,13,19.22,0.0,690.0,694.0,15.0,0.0,0.0,27176.0,46.0,21.0,1,0,Apr-1994,0.0,1.0,0.0,7.0,13.0,13.0,7.0,7.0,2.0,13.0,17.0,11.0,15.0,None,0.0,0.0,6.0
9,9,6500.0,3,10.99,212.78,B,B4,54.0,None,1,15300.0,2,2015-11-01,0,0,59.0,11,24.39,0.0,725.0,729.0,7.0,0.0,0.0,2936.0,30.6,63.0,0,0,Jan-1993,0.0,1.0,0.0,4.0,5.0,5.0,4.0,21.0,24.0,6.0,39.0,5.0,7.0,0.0,0.0,0.0,8.0


#### Define columns

In [4]:
label_colunms = ['isDefault']
special_colunms = ['id']
continuous_colunms = ['loanAmnt', 'interestRate', 'installment', 'employmentTitle', 'annualIncome', 'postCode', 'dti', 
                      'delinquency_2years', 'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec', 'pubRecBankruptcies',
                      'revolBal', 'revolUtil', 'totalAcc', 'title', 'policyCode'] 
categorical_colunms = [c for c in train_dataset.columns if c not in label_colunms + special_colunms + continuous_colunms]
colunms_to_woe = [c for c in train_dataset.columns if c not in label_colunms + special_colunms]

#### Handle missing data

In [5]:
train_dataset = train_dataset.na.fill("0.0", subset=continuous_colunms)
train_dataset = train_dataset.na.fill("Empty", subset=categorical_colunms)

train_dataset = train_dataset.select(*(F.col(c).cast('double').alias(c) if c in continuous_colunms + label_colunms else F.col(c) for c in train_dataset.columns))

#### Convert continuous features to categorical

In [6]:
df_bucketed = quantile_bucket(train_dataset, continuous_colunms, 10)
df_bucketed = df_bucketed.select(*train_dataset.columns)
df_bucketed.limit(10).toPandas()

22/06/10 07:52:31 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 3 buckets as a result.
22/06/10 07:52:31 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 3 buckets as a result.
22/06/10 07:52:31 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 3 buckets as a result.
22/06/10 07:52:31 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 6 buckets as a result.
22/06/10 07:52:31 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 2 buckets as a result.


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,9.0,5,9.0,9.0,E,E2,2.0,2 years,2,8.0,2,2014-07-01,1.0,1,3.0,32,4.0,1.0,8.0,8.0,2.0,1.0,1.0,8.0,4.0,6.0,0,0,Aug-2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,7.0,5,8.0,6.0,D,D2,8.0,5 years,0,2.0,2,2012-08-01,0.0,0,3.0,18,8.0,1.0,6.0,6.0,7.0,1.0,1.0,6.0,3.0,3.0,1,0,May-2002,5.0,1.0,Empty,Empty,Empty,Empty,10.0,Empty,Empty,Empty,Empty,Empty,13.0,Empty,Empty,Empty,Empty
2,2,5.0,5,8.0,3.0,D,D3,6.0,8 years,0,6.0,2,2015-10-01,0.0,0,6.0,14,7.0,1.0,3.0,3.0,5.0,1.0,1.0,1.0,4.0,6.0,0,0,May-2006,1.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,4.0,3,0.0,4.0,A,A4,6.0,10+ years,1,8.0,1,2015-08-01,0.0,4,3.0,11,4.0,1.0,4.0,4.0,4.0,1.0,1.0,4.0,5.0,6.0,1,0,May-1999,3.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,0.0,3,5.0,0.0,C,C2,1.0,Empty,1,0.0,2,2016-03-01,0.0,10,6.0,21,9.0,1.0,5.0,5.0,6.0,1.0,1.0,0.0,2.0,6.0,0,0,Aug-1977,4.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0
5,5,4.0,3,1.0,4.0,A,A5,6.0,7 years,0,1.0,2,2017-04-01,0.0,9,8.0,21,4.0,1.0,8.0,8.0,9.0,1.0,1.0,1.0,2.0,9.0,1,0,Jul-1998,4.0,1.0,12.0,1.0,2.0,2.0,1.0,1.0,48.0,2.0,3.0,2.0,19.0,0.0,0.0,0.0,0.0
6,6,0.0,3,1.0,0.0,A,A4,8.0,9 years,0,1.0,0,2014-10-01,0.0,0,8.0,14,4.0,1.0,9.0,9.0,6.0,1.0,1.0,1.0,0.0,5.0,0,0,Oct-2006,1.0,1.0,0.0,1.0,3.0,3.0,7.0,11.0,3.0,10.0,18.0,3.0,12.0,0.0,0.0,0.0,3.0
7,7,4.0,3,6.0,5.0,C,C3,8.0,1 year,1,0.0,2,2014-01-01,0.0,0,2.0,4,9.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,7.0,1,0,Dec-1994,1.0,1.0,0.0,4.0,4.0,4.0,4.0,16.0,10.0,5.0,21.0,4.0,8.0,0.0,0.0,0.0,2.0
8,8,5.0,3,5.0,5.0,C,C2,1.0,5 years,2,4.0,1,2016-05-01,1.0,0,9.0,13,5.0,1.0,5.0,5.0,8.0,1.0,1.0,8.0,4.0,4.0,1,0,Apr-1994,1.0,1.0,0.0,7.0,13.0,13.0,7.0,7.0,2.0,13.0,17.0,11.0,15.0,Empty,0.0,0.0,6.0
9,9,2.0,3,3.0,1.0,B,B4,1.0,Empty,1,0.0,2,2015-11-01,0.0,0,1.0,11,7.0,1.0,8.0,8.0,2.0,1.0,1.0,0.0,2.0,9.0,0,0,Jan-1993,1.0,1.0,0.0,4.0,5.0,5.0,4.0,21.0,24.0,6.0,39.0,5.0,7.0,0.0,0.0,0.0,8.0


#### WoE encoding

In [7]:
df_woe = woe_encode(df_bucketed, colunms_to_woe, 'isDefault', 1.0)
df_woe.limit(10).toPandas()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,loanAmnt_woe,term_woe,interestRate_woe,installment_woe,grade_woe,subGrade_woe,employmentTitle_woe,employmentLength_woe,homeOwnership_woe,annualIncome_woe,verificationStatus_woe,issueDate_woe,purpose_woe,postCode_woe,regionCode_woe,dti_woe,delinquency_2years_woe,ficoRangeLow_woe,ficoRangeHigh_woe,openAcc_woe,pubRec_woe,pubRecBankruptcies_woe,revolBal_woe,revolUtil_woe,totalAcc_woe,initialListStatus_woe,applicationType_woe,earliesCreditLine_woe,title_woe,policyCode_woe,n0_woe,n1_woe,n2_woe,n3_woe,n4_woe,n5_woe,n6_woe,n7_woe,n8_woe,n9_woe,n10_woe,n11_woe,n12_woe,n13_woe,n14_woe
0,0,9.0,5,9.0,9.0,E,E2,2.0,2 years,2,8.0,2,2014-07-01,1.0,1,3.0,32,4.0,1.0,8.0,8.0,2.0,1.0,1.0,8.0,4.0,6.0,0,0,Aug-2001,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,0.212384,0.652008,0.980909,0.179946,0.917968,0.886628,-0.035858,0.003257,0.051106,-0.225376,0.224884,-0.070409,0.519035,-0.026490,0.075052,-0.097408,-0.023223,-0.438466,-0.438466,-0.074541,-0.037466,-0.025199,-0.029686,0.014164,-0.027039,0.015550,-0.006394,0.021583,0.120767,0.0,-0.013497,-0.091526,-0.225819,-0.225819,-0.009372,-0.012631,0.024390,-0.101023,-0.005999,-0.220500,-0.055832,-0.001482,0.016079,0.007389,0.002365
1,1,7.0,5,8.0,6.0,D,D2,8.0,5 years,0,2.0,2,2012-08-01,0.0,0,3.0,18,8.0,1.0,6.0,6.0,7.0,1.0,1.0,6.0,3.0,3.0,1,0,May-2002,5.0,1.0,Empty,Empty,Empty,Empty,10.0,Empty,Empty,Empty,Empty,Empty,13.0,Empty,Empty,Empty,Empty,0.150643,0.652008,0.613972,0.066927,0.560321,0.530459,-0.239357,-0.023787,-0.185441,0.126109,0.224884,-0.238212,0.072657,-0.026490,-0.274387,0.292001,-0.023223,-0.040004,-0.040004,0.028316,-0.037466,-0.025199,0.047673,-0.031812,0.015964,-0.021991,-0.006394,0.110939,-0.335386,0.0,-0.349841,-0.349841,-0.349841,-0.349841,0.088785,-0.349841,-0.349841,-0.349841,-0.349871,-0.349841,0.041934,0.015512,-0.349841,-0.349841,-0.349841
2,2,5.0,5,8.0,3.0,D,D3,6.0,8 years,0,6.0,2,2015-10-01,0.0,0,6.0,14,7.0,1.0,3.0,3.0,5.0,1.0,1.0,1.0,4.0,6.0,0,0,May-2006,1.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,0.040077,0.652008,0.613972,-0.023396,0.560321,0.561095,0.004855,-0.014380,-0.185441,-0.063052,0.224884,-0.051211,0.072657,0.004208,-0.014202,0.163994,-0.023223,0.230524,0.230524,0.026030,-0.037466,-0.025199,-0.005503,0.014164,-0.027039,0.015550,-0.006394,0.126182,0.120767,0.0,-0.013497,0.005277,-0.154578,-0.154578,0.099030,0.380758,0.066614,-0.101023,0.051932,-0.148058,0.042338,-0.001482,0.016079,0.007389,0.261988
3,3,4.0,3,0.0,4.0,A,A4,6.0,10+ years,1,8.0,1,2015-08-01,0.0,4,3.0,11,4.0,1.0,4.0,4.0,4.0,1.0,1.0,4.0,5.0,6.0,1,0,May-1999,3.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,0.021457,-0.268618,-1.585257,0.111268,-1.355566,-1.240844,0.004855,-0.080023,0.192890,-0.225376,0.060872,-0.011346,-0.201407,-0.026490,0.074603,-0.097408,-0.023223,0.135115,0.135115,-0.021826,-0.037466,-0.025199,0.031703,0.069930,-0.027039,-0.021991,-0.006394,-0.003766,-0.149019,0.0,0.003692,0.029135,0.044226,0.044226,-0.009372,-0.075491,0.016291,-0.013259,0.007773,0.051284,-0.002343,-0.001482,0.016079,0.007389,-0.148513
4,4,0.0,3,5.0,0.0,C,C2,1.0,Empty,1,0.0,2,2016-03-01,0.0,10,6.0,21,9.0,1.0,5.0,5.0,6.0,1.0,1.0,0.0,2.0,6.0,0,0,Aug-1977,4.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,-0.311320,-0.268618,0.028718,-0.410185,0.152694,0.045582,0.196091,0.388563,0.192890,0.233323,0.224884,0.108472,0.179074,0.004208,0.096883,0.492354,-0.023223,0.039023,0.039023,0.018438,-0.037466,-0.025199,-0.007056,-0.101815,-0.027039,0.015550,-0.006394,0.176897,-0.045973,0.0,0.132020,-0.091526,0.100124,0.100124,-0.028484,0.06054